# Preprocessing

In [12]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion formant frequencies

In [13]:
import numpy as np
from scipy.signal import lfilter
from scipy.linalg import toeplitz

def calculate_formants(audio_signal, sample_rate=16000, lpc_order=12, num_formants=3):
    """
    Berechnet die ersten Formantfrequenzen eines Audiosignals mit LPC-Analyse.

    Quelle:
        Markel, J. D., & Gray, A. H. (1976). Linear Prediction of Speech.
        Springer-Verlag.

    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        sample_rate (int): Sampling-Rate des Signals (Standard: 16000 Hz).
        lpc_order (int): Ordnung des LPC-Modells (Standard: 12).
        num_formants (int): Anzahl der zu extrahierenden Formanten (Standard: 3).

    Returns:
        np.array: Die geschätzten Formant-Frequenzen (in Hz), gefiltert von 0 Hz-Werten.
    """
    # Berechnung der Autokorrelation
    autocorr = np.correlate(audio_signal, audio_signal, mode='full')
    autocorr = autocorr[len(autocorr) // 2:]  # Nur positive Hälfte verwenden

    # Yule-Walker-Gleichung lösen, um LPC-Koeffizienten zu berechnen
    R = toeplitz(autocorr[:lpc_order])
    r = autocorr[1:lpc_order+1]
    
    try:
        lpc_coeffs = np.linalg.solve(R, r)
    except np.linalg.LinAlgError:
        return np.array([])  # Falls Matrix singulär ist (z. B. Stille), keine Formanten zurückgeben

    # Berechnung der Polstellen des LPC-Filters
    roots = np.roots(np.concatenate([[1], -lpc_coeffs]))  # LPC-Koeffizienten als Filter nutzen
    roots = [r for r in roots if np.imag(r) >= 0]  # Nur positive Frequenzen behalten

    # Umwandlung in Frequenzen
    formant_frequencies = np.angle(roots) * (sample_rate / (2 * np.pi))

    # Entferne unrealistische Formanten (z. B. 0 Hz)
    formant_frequencies = np.array([f for f in formant_frequencies if f > 50])  # Formanten unter 50 Hz ignorieren

    # Sortieren und die ersten num_formants Formanten zurückgeben
    formant_frequencies = np.sort(formant_frequencies)[:num_formants]

    return formant_frequencies

# Test

In [14]:
import os

# Formanten für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)

        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)

        # Formanten berechnen
        formant_values = calculate_formants(audio_signal)

        print(f"Formant Frequencies ({file_name}): {formant_values}")

Formant Frequencies (_noise_pink.wav): [ 355.82010688 2047.20185842 3588.03598255]
Formant Frequencies (_noise_white.wav): [1602.84057166 3107.72120139 4419.59238914]
Formant Frequencies (_signal_constant.wav): [1530.42740933 3002.39925651 4442.72974032]
Formant Frequencies (_signal_silence.wav): []
Formant Frequencies (_signal_sine.wav): [ 439.98976024  760.46543095 2295.07770464]
Formant Frequencies (_snr_03.wav): [ 365.82032734 1071.25969598 2070.03260883]
Formant Frequencies (_snr_10.wav): [ 618.48467439 1470.45761424 2351.76105456]
Formant Frequencies (_snr_20.wav): [ 628.31128474 1477.19639622 2367.7985499 ]
Formant Frequencies (example1.wav): [ 158.59468638 1780.55449126 3886.47694109]
Formant Frequencies (example2.wav): [ 141.6326966  1405.22350924 3517.14368178]
Formant Frequencies (example3.wav): [ 168.80719644  711.08047451 3377.51504226]
